# Experiment Chooser
This widget will allow you to select an experiment directory and persist your choice across notebook boundaries. That means you only need to select your experiment once even if you're running analyses in multiple notebooks.

import this namespace with the magic command `%run utils/experiment_chooser.ipynb`

In [ ]:
from ipyfilechooser import FileChooser
from improc.experiment import loader

import pathlib

class experiment_chooser:
    def fetch_base() -> str:
        %store -r experiment_base
        return experiment_base

    def fetch():
        %store -r experiment
        return experiment

    def choose():
        try:
            import config.local as config
        except:
            import config.default as config

        def verify_experiment_dir(chooser):
            path = pathlib.Path(fc.selected_path)
            scratch = config.SCRATCH_PATH / path.name
            try:
                # magic command resolution is weird so we have to define these vars as globals and use them outside the try/except
                global experiment; experiment = loader.load_experiment(path, scratch).value
                print(experiment)
                global experiment_base; experiment_base = path
                chooser.title = ""
            except Exception as e:
                chooser.title = f'<b style="color:red">Error: {e}</b>'
            %store experiment
            %store experiment_base

        fc = FileChooser(config.EXPERIMENT_DIRECTORY, show_only_dirs=True)
        fc.title = "<b>Select experiment directory</b>"
        fc.register_callback(verify_experiment_dir)

        try:
            %store -r experiment_base
            fc.default_path = experiment_base
            fc.title = ""
            fc._apply_selection()
        except NameError:
            pass

        display(fc)